In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import os as os
import pandas as pd
import re
keywordlist=["health","education"]
max_num=5
# datasetsname:[tag1,tag2]
datasets_with_tag_array={}
# ready for treemap; contains keyword as name and tags as children {"name"=keyword,"value"=75,children=[]} 
keyword_and_tag_array=[]
# To avoid duplicate tags;
uniq_taglist=[]
for keyword in keywordlist:
    #array to hold dictionaries containing tag name and value [{"name":tagname, "value":25},{"name":tagname2, "value":25}]
    temp_dict_for_datasets_and_tag=[]
    # temoporay dictionary to hold name value and children
    temp_dict={}
    count=0
    # list that contains all the links from first page
    link_list=[]
    first_url="https://data.cityofnewyork.us/browse?amp=&q="+keyword+"&sortBy=relevance&page=1"
    link_list.append(first_url)
    paginationlink = requests.get(first_url)
    soup = BeautifulSoup(paginationlink.text, "html.parser")
    links=soup.findAll("a",{"class":"pageLink"})
    for link in links:
        link_list.append("https://data.cityofnewyork.us"+link['href'])
    print(link_list)
    for url in link_list:
        if(count<max_num):
            datalink1="https://dev.socrata.com/foundry/data.cityofnewyork.us/"
            datalink2='https://data.cityofnewyork.us/resource/'
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            # parent div with meta tag
            links=soup.findAll("div", {"class": "browse2-result"})
            dict2={}
            #print("url in link")
        if(count<max_num):
            count2=0
            for link in links:
                tags_array=[]
                array_to_contain_tag_dict=[]
                if(count2<max_num):
                    try:
                        # get href and add .json at the end
                        href2=link.findAll("a",{"class":"browse2-result-api-link"})[0]["href"].replace(datalink1,datalink2)
                        href=href2+".json"
                        # Get the tags from here
                        #print("link is : ",href2)
                        title=link.findAll("a",{"class":"browse2-result-name-link"})[0].string
                        dict2[title]=href
                        taglist=link.findAll("div",{"class":"browse2-result-topics"})
                        tag_arr=taglist[0].findAll("a")
                        for tag_element in tag_arr:
                            tag=tag_element.find("span").string
                            tags_array.append(tag)
                            if tag not in uniq_taglist:
                                uniq_taglist.append(tag)
                                array_to_contain_tag_dict.append({"name":tag,"value":25})
                        datasets_with_tag_array[title]=tags_array
                        temp_dict_for_datasets_and_tag.extend(array_to_contain_tag_dict)
                        count2=count2+1
                    except IndexError:
                        pass
                        #print(link.findAll("a",{"class":"browse2-result-name-link"})[0].string)
            #print(dict2)
        for key in dict2:
            if(count<max_num):
                try:
                    data=pd.read_json(dict2[key])
                    #print(key)
                    if not (os.path.exists("upload/"+re.sub("/","",key)+".csv")):
                        data.to_csv(r"upload/"+re.sub("/","",key)+".csv",sep=",")
                        count=count+1
                        print("fetched: ",count,"\n")
                except:
                    print("error in file location")
    temp_dict["name"]=keyword
    temp_dict["value"]=75
    temp_dict["children"]=temp_dict_for_datasets_and_tag
    keyword_and_tag_array.append(temp_dict)
    

['https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=2', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=3', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=4', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=5', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=6', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=7', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=8', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=9']
fetched:  1 

fetched:  2 

fetched:  3 

fetched:  4 

fetched:  5 

['https://data.cityofnewyork.us/browse?amp=&q=education&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=education&sortBy=relev